In [1]:
#### Team 4 Project - GEDCOM data
##### Fariha Rahman
##### Siddique Baksh
##### Oussama Elhaddad


In [5]:
pip install python-gedcom


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tabulate


  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 37.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install numpy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
### gedcom library

from gedcom.parser import Parser
from gedcom.element.individual import IndividualElement
from gedcom.element.family import FamilyElement
from tabulate import tabulate
import pandas as pd
import numpy as np
import datetime
from datetime import date

import warnings
warnings.filterwarnings("ignore")

In [10]:
def calculateAge(birthDate, deathDate):
    
      age = deathDate.year - birthDate.year - ((deathDate.month, deathDate.day) <
            (birthDate.month, birthDate.day))
         
 
      return age

In [14]:
def getIndividuals(file):
    
    ### dataframe to store individuals details
    individuals = pd.DataFrame(columns=['ID','Name','Gender','Birthday','Age','Alive','Death','Child','Spouse'])
    
    ### Parse the gedcom file
    parser = Parser()
    parser.parse_file(file)
    
    ### Iterate through each element and pull an individual data based on level and tag conditions
    for ele in parser.get_element_list():
        
        if(IndividualElement.is_individual(ele) == True):
            
            famSpouses = set()
            famChilds = set()            
            id = ele.get_pointer().replace('@', '')
            deathDate = 'NA'
            alive = 'True'
            
            ### For each individual element, below extracting the personal data
            child_elements = IndividualElement.get_child_elements(ele)
            for child in child_elements:
                if(child.get_tag() == 'NAME'):
                    name = child.get_value()
                    # print(name)
                elif(child.get_tag() == 'SEX'):
                    gender = child.get_value()
                    # print(gender)
                elif(child.get_tag() == 'BIRT'):
                    birthDate = datetime.datetime.strptime(child.get_child_elements()[0].get_value(),'%d %b %Y').strftime('%Y-%m-%d')
                    # print(birthDate)
                    age = calculateAge(date(int(birthDate.split('-')[0]),int(birthDate.split('-')[1]),int(birthDate.split('-')[2])), 
                            date.today())
                elif(child.get_tag() == 'FAMS'):
                    famSpouses.add(child.get_value().replace('@', ''))
                    # print('Spouse ', famSpouses)
                elif(child.get_tag() == 'FAMC'):
                    famChilds.add(child.get_value().replace('@', ''))
                    # print('Childs ', famChilds)
                elif(child.get_tag() == 'DEAT'):
                    deathDate = datetime.datetime.strptime(child.get_child_elements()[0].get_value(),'%d %b %Y').strftime('%Y-%m-%d')
                    # print(deathDate)
                    alive = 'False'
                    age = calculateAge(date(int(birthDate.split('-')[0]),int(birthDate.split('-')[1]),int(birthDate.split('-')[2])), 
                           date(int(deathDate.split('-')[0]),int(deathDate.split('-')[1]),int(deathDate.split('-')[2])))
                    
        
            # print('*********')  
        ### Appending each individual to dataframe
            row = {'ID': id, 'Name': name, 'Gender': gender, 
                'Birthday': birthDate, 'Age': age, 'Alive': alive,
                'Death': deathDate, 'Child': famChilds, 'Spouse': famSpouses}
            
            individuals = individuals.append(row, ignore_index = True)
            
    return individuals

In [15]:
def getFamilies(file):
    
    ### dataframe to store individuals details
    families = pd.DataFrame(columns=['ID','Married','Divorced','Husband ID','Husband Name','Wife ID','Wife Name','Children'])
    
    ### Parse the gedcom file
    parser = Parser()
    parser.parse_file(file)
    
    for ele in parser.get_element_list():

        if(FamilyElement.is_family(ele) == True):
                
                childrens = set()
                id = ele.get_pointer().replace('@', '')
                divorce = 'NA'
                childs = IndividualElement.get_child_elements(ele)
                for child in childs:
                    
                    if(child.get_tag() == 'HUSB'):
                        husbID = child.get_value().replace('@', '')
                    elif(child.get_tag() == 'WIFE'):
                        wifeID = child.get_value().replace('@', '')
                    elif(child.get_tag() == 'CHIL'):
                        childrens.add(child.get_value().replace('@', ''))
                    elif(child.get_tag() == 'MARR'):
                        marriage = IndividualElement.get_child_elements(child)[0].get_value()
                    elif(child.get_tag() == 'DIV'):
                        divorce = IndividualElement.get_child_elements(child)[0].get_value()
                        
                individuals = getIndividuals(file)
                
                husbName = individuals.loc[individuals['ID'] == husbID, 'Name'].iloc[0]
                wifeName = individuals.loc[individuals['ID'] == wifeID, 'Name'].iloc[0]
                ### Appending each individual to dataframe
                row = {'ID': id, 'Married': marriage, 'Divorced': divorce, 
                    'Husband ID': husbID, 'Husband Name': husbName, 'Wife ID': wifeID,
                    'Wife Name': wifeName, 'Children': childrens}
                
                families = families.append(row, ignore_index = True)
    
    return families

